# PeruCovid19 & LatamCovid19 & DSRP

### 1.- Loading data


Deleting all before load data

In [0]:
# To delete last directory
!chmod 777 /content/
!rm -rf '/content/covid-19_latinoamerica/'
%reset #Clear variables

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Don't know how to reset  #clear, please run `%reset?` for details
Don't know how to reset  variables, please run `%reset?` for details


In [0]:
# Clone repository
!git clone https://github.com/DataScienceResearchPeru/covid-19_latinoamerica.git

Cloning into 'covid-19_latinoamerica'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (253/253), done.
remote: Total 4549 (delta 250), reused 101 (delta 62), pack-reused 4234
Receiving objects: 100% (4549/4549), 29.47 MiB | 20.93 MiB/s, done.
Resolving deltas: 100% (3226/3226), done.


In [0]:
# Importing libraries
import pandas as pd
import numpy as np
import datetime
import os
import time

path='/content/covid-19_latinoamerica/latam_covid_19_data/daily_reports/'
path, dirs, files = next(os.walk(path))
numero_archivos = len(files) 
print('Hay {} archivos en el folder y 1 el README. Iteraremos {} veces'.format(numero_archivos,numero_archivos-1))

Hay 64 archivos en el folder y 1 el README. Iteraremos 63 veces


Loading ISO 3166-2 Code

In [0]:
url='https://raw.githubusercontent.com/DataScienceResearchPeru/covid-19_latinoamerica/master/utils/iso3166-2.csv'
template_isocode=pd.read_csv(url, sep=',')
isocode=template_isocode['Code']
isocode=isocode[isocode!='PE-LMA']
array_isocode=isocode.to_numpy()

Generating array with name of files

In [0]:
# Generamos un arreglo con cada día que se tiene en archivo.
base = (datetime.datetime.today()).date()
numdays = numero_archivos-1
date_list = [str(base - datetime.timedelta(days=x))+str('.csv') for x in range(numdays)]
print('Agregamos {} nombres de archivo al arreglo'.format(len(date_list)))
str(date_list)

Agregamos 63 nombres de archivo al arreglo


"['2020-04-27.csv', '2020-04-26.csv', '2020-04-25.csv', '2020-04-24.csv', '2020-04-23.csv', '2020-04-22.csv', '2020-04-21.csv', '2020-04-20.csv', '2020-04-19.csv', '2020-04-18.csv', '2020-04-17.csv', '2020-04-16.csv', '2020-04-15.csv', '2020-04-14.csv', '2020-04-13.csv', '2020-04-12.csv', '2020-04-11.csv', '2020-04-10.csv', '2020-04-09.csv', '2020-04-08.csv', '2020-04-07.csv', '2020-04-06.csv', '2020-04-05.csv', '2020-04-04.csv', '2020-04-03.csv', '2020-04-02.csv', '2020-04-01.csv', '2020-03-31.csv', '2020-03-30.csv', '2020-03-29.csv', '2020-03-28.csv', '2020-03-27.csv', '2020-03-26.csv', '2020-03-25.csv', '2020-03-24.csv', '2020-03-23.csv', '2020-03-22.csv', '2020-03-21.csv', '2020-03-20.csv', '2020-03-19.csv', '2020-03-18.csv', '2020-03-17.csv', '2020-03-16.csv', '2020-03-15.csv', '2020-03-14.csv', '2020-03-13.csv', '2020-03-12.csv', '2020-03-11.csv', '2020-03-10.csv', '2020-03-09.csv', '2020-03-08.csv', '2020-03-07.csv', '2020-03-06.csv', '2020-03-05.csv', '2020-03-04.csv', '2020-03

In [0]:
# Convertimos a data frame cada archivo.
array_dataframes=[]
indice=0
print('En total hay {} archivos'.format(numdays))
for date in date_list:
  array_dataframes.append(pd.read_csv(path+date, sep=','))    
  array_dataframes[indice]['Date']=date[:-4]
  #del array_dataframes[indice]['Last Update'] 
  #del array_dataframes[indice]['Country']
  #del array_dataframes[indice]['Subdivision']
  # Crear dataframe total
  indice+=1

En total hay 63 archivos


In [0]:
total_data=pd.DataFrame() # structure 
for m in range(len(array_dataframes)-1):
  total_data=pd.concat([total_data,array_dataframes[m]], ignore_index=True)  
# Eliminamos duplicados
total_data=total_data.drop_duplicates(keep=False)

Load first cases


In [0]:
url='https://raw.githubusercontent.com/DataScienceResearchPeru/covid-19_latinoamerica/7d6255612022dec1b005e6c040a7b9c2bf70b852/utils/first_cases_date.csv'
template_first_cases=pd.read_csv(url, sep=',')
first_cases=template_first_cases[['ISO 3166-2 Code','First Case Date']]

Checking errors

In [0]:
def missing_data_checker(iteracion,mode,array_pnf,df):
  i=iteracion
  m=mode
  a=array_pnf[0]
  b=array_pnf[1]
  c=array_pnf[2]
  result='error'
  today=datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%s')[:-8]
  array_result=[0,
                'pass', #1
                'future', #2
                'pass', #3
                'now', #4
                'now_future', #5
                'now', #6
                'pass', #7
                'future', #8
                'pass', #9
                'pass', #10
                'future', #11
                'pass', #12
                'pass', #13
                'pass', #14
                'pass', #15
                'pass', #16
                'future', #17
                'pass', #18
                'pass', #19
                'future', #20
                'pass', #21
                'now', #22
                'now_future', #23
                'now', #24
                'pass', #25
                'future', #26
                'pass', #27
                ]

  if (a==''):
    if (b==''):
      if (c==''): #1
        result=array_result[1]
      elif (c==0): #2
        result=array_result[2]
      else: # (c>0): #3
        result=array_result[3]
    elif (b==0):
      if (c==''): #4
        result=array_result[4]
      elif (c==0): #5
        result=array_result[5]
      else: # (c>0): #6
        result=array_result[6]
    else: #(b>0):
      if (c==''): #7
        result=array_result[7]
      elif (c==0): #8
        result=array_result[8]
      else: # (c>0): #9
        result=array_result[9]
  elif (a==0):
    if (b==''):
      if (c==''): #10
        result=array_result[10]
      elif (c==0): #11
        result=array_result[11]
      else: # (c>0): #12
        result=array_result[12]
    elif (b==0):
      if (c==''): #13
        result=array_result[13]
      elif (c==0): #14
        result=array_result[14]
      else: # (c>0): #15
        result=array_result[15]
    else: # (b>0):
      if (c==''): #16
        result=array_result[16]
      elif (c==0): #17
        result=array_result[17]
      else: # (c>0): #18
        result=array_result[18]   
  else: # (a>0):
    if (b==''):
      if (c==''): #19
        result=array_result[19]
      elif (c==0): #20
        result=array_result[20]
      else: # (c>0): #21
        result=array_result[21]
    elif (b==0):
      if (c==''): #22
        result=array_result[22]
      elif (c==0): #23
        result=array_result[23]
      else: # (c>0): #24
        result=array_result[24]
    else: # (b>0):
      if (c==''): #25
        result=array_result[25]
      elif (c==0): #26
        result=array_result[26]
      else: # (c>0): #27
        result=array_result[27]       

  flag_counter=0  
  # Uncomment this to see errors
  if result == 'pass':
    pass
  elif result!='error':
    if result=='future':
      df.loc[i+1,m]=str('')
      df.loc[i+1,'Last Update']=today

    if result=='now': 
      df.loc[i,m]=str('')
      df.loc[i,'Last Update']=today
      
    if result=='now_future': 
      df.loc[i,m]=str('')
      df.loc[i,'Last Update']=today
      df.loc[i+1,m]=str('')
      df.loc[i+1,'Last Update']=today
    
    flag_counter+=1
    #print('a:{} b:{} c:{} resultado:{}'.format(a,b,c,result))      
  else:
    print('WARNING! - result={}'.format(result),end='')
      
    

  # errors_confirmed.loc[index]=[str(code),str(day),str(past_confirmed),str(init_confirmed),str(Future_confirmed)]
  return [df,flag_counter]

In [0]:
def first_cases_zeros(df,code):
  df['Date']=pd.to_datetime(df['Date'],format='%Y-%m-%d')
  z=first_cases[first_cases['ISO 3166-2 Code']==code]['First Case Date'].values[0] # First date as value
  x=df['Date']<datetime.datetime.strptime(z,'%Y-%m-%d')
  y=['Confirmed','Deaths','Recovered']
  df.loc[x,y]=0 # Boolean, Column
  # Last Update
  today=datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%s')[:-8]
  b=['Last Update']
  df.loc[x,b]=today
  return df

In [0]:
fixed_data = []

for code in array_isocode:
    # NaN to ''
    temp = total_data[total_data['ISO 3166-2 Code'] == code]
    df = temp.copy()  # Delete warning
    df = df.fillna({'ISO 3166-2 Code': '',
                    'Country': '',
                    'Subdivision': '',
                    'Last Update': '',
                    'Confirmed': '',
                    'Deaths': '',
                    'Recovered': ''})
    df = df.sort_values(by=['Date'])
    df = df.reset_index(drop=True)
    df = first_cases_zeros(df, code)  # Fill zeros at the beginning

    counter_flags = 100
    while counter_flags != 0:
      
        for iteration in range(len(date_list)-2):
            # We skip first and last iteration because we have no data for
            # past and future these days
            time.sleep(0.05)
            if iteration == 0:
                continue
            elif iteration > len(date_list)-4:
                pass
            else:
                #print(iteration, end="")
                # Past
                Past_confirmed = int(float('0'+str(df.loc[iteration-1]['Confirmed'])))
                Past_deaths = int(float('0'+str(df.loc[iteration-1]['Deaths'])))
                Past_recovered = int(float('0'+str(df.loc[iteration-1]['Recovered'])))
                # Now
                Now_confirmed = int(float('0'+str(df.loc[iteration]['Confirmed'])))
                Now_deaths = int(float('0'+str(df.loc[iteration]['Deaths'])))
                Now_recovered = int(float('0'+str(df.loc[iteration]['Recovered'])))
                # Future
                Future_confirmed = int(float('0'+str(df.loc[iteration+1]['Confirmed'])))
                Future_deaths = int(float('0'+str(df.loc[iteration+1]['Deaths'])))
                Future_recovered = int(float('0'+str(df.loc[iteration+1]['Recovered'])))

                # Compare day after vs before in confirmed
                conf = [Past_confirmed, Now_confirmed, Future_confirmed]
                deat = [Past_deaths, Now_deaths, Future_deaths]
                reco = [Past_recovered, Now_recovered, Future_recovered]

                df, fc = missing_data_checker(iteration, 'Confirmed', conf, df)
                df, fd = missing_data_checker(iteration, 'Deaths', deat, df)
                df, fr = missing_data_checker(iteration, 'Recovered', reco, df)

                counter_flags += (fc+fd+fr)
            # endif
        # endfor
        #print('Counter_flags:{}'.format(counter_flags))
        counter_flags = 0
    # endwhile
    fixed_data.append(df)
    print('Subdivision:{} '.format(code))
  

Subdivision:AR-B 
Subdivision:AR-K 
Subdivision:AR-H 
Subdivision:AR-U 
Subdivision:AR-C 
Subdivision:AR-W 
Subdivision:AR-X 
Subdivision:AR-E 
Subdivision:AR-P 
Subdivision:AR-Y 
Subdivision:AR-L 
Subdivision:AR-F 
Subdivision:AR-M 
Subdivision:AR-N 
Subdivision:AR-Q 
Subdivision:AR-R 
Subdivision:AR-A 
Subdivision:AR-J 
Subdivision:AR-D 
Subdivision:AR-Z 
Subdivision:AR-S 
Subdivision:AR-G 
Subdivision:AR-V 
Subdivision:AR-T 
Subdivision:BO-H 
Subdivision:BO-C 
Subdivision:BO-B 
Subdivision:BO-L 
Subdivision:BO-O 
Subdivision:BO-N 
Subdivision:BO-P 
Subdivision:BO-S 
Subdivision:BO-T 
Subdivision:BR-AC 
Subdivision:BR-AL 
Subdivision:BR-AP 
Subdivision:BR-AM 
Subdivision:BR-BA 
Subdivision:BR-CE 
Subdivision:BR-DF 
Subdivision:BR-ES 
Subdivision:BR-GO 
Subdivision:BR-MA 
Subdivision:BR-MT 
Subdivision:BR-MS 
Subdivision:BR-MG 
Subdivision:BR-PR 
Subdivision:BR-PB 
Subdivision:BR-PA 
Subdivision:BR-PE 
Subdivision:BR-PI 
Subdivision:BR-RJ 
Subdivision:BR-RN 
Subdivision:BR-RS 
Subdivi

Agroup per Date all fixed data

In [0]:
total_data_fixed=pd.DataFrame() # structure 
for m in range(len(fixed_data)):
  total_data_fixed=pd.concat([total_data_fixed,fixed_data[m]], ignore_index=True)  
# Eliminamos duplicados
total_data_fixed=total_data_fixed.drop_duplicates(keep=False)
# Reorder
total_data_fixed=total_data_fixed.sort_values(by=['Date','Country','Subdivision'])

### 3.-Updating data

Change missing data to blank spaces instead using zeros with no-sense

In [0]:
def find_country_name(row):
  # index of ISO CODE
  x=row['ISO 3166-2 Code']
  m=template_isocode[template_isocode['Code']==x].index.values[0]  
  country_name=template_isocode.loc[m]['Country Name']
  return country_name

def find_subdivision_name(row):
  # index of ISO CODE
  x=row['ISO 3166-2 Code']
  m=template_isocode[template_isocode['Code']==x].index.values[0]  
  subdivision_name=template_isocode.loc[m]['Subdivision Name Used']
  return subdivision_name

array_total_data_fixed=[]
for date in date_list:
  temp=total_data_fixed[total_data_fixed['Date']==date[:-4]]
  temp_day=temp[['ISO 3166-2 Code','Country','Subdivision','Last Update','Confirmed','Deaths','Recovered']].copy()
  array_total_data_fixed.append(temp_day)

Fix format

In [0]:
for a in range(len(date_list)):
  array_total_data_fixed[a]['Confirmed']=array_total_data_fixed[a]['Confirmed'].astype(str).astype(float,errors='ignore').astype(int,errors='ignore')
  array_total_data_fixed[a]['Deaths']=array_total_data_fixed[a]['Deaths'].astype(str).astype(float,errors='ignore').astype(int,errors='ignore')
  array_total_data_fixed[a]['Recovered']=array_total_data_fixed[a]['Recovered'].astype(str).astype(float,errors='ignore').astype(int,errors='ignore')
  #array_total_data_fixed[a]=array_total_data_fixed[a].applymap(str)

In [0]:
array_total_data_fixed[14]

In [0]:
today=datetime.date.today()
path2='/content/covid-19_latinoamerica/latam_covid_19_data/daily_reports_fixed/'
!mkdir '/content/covid-19_latinoamerica/latam_covid_19_data/daily_reports_fixed/'

for number_date in range(date_list):
  string_file='{}{}.csv'.format(path2,date[:-4])
  array_total_data_fixed[number_date].to_csv(string_file, sep=',',index=False)
  print('Generated {}'.format(string_file))

In [0]:
# Download all in the folder
!zip -r /content/daily_report.zip /content/covid-19_latinoamerica/latam_covid_19_data/daily_reports_fixed

In [0]:
# Download the data
from google.colab import files
files.download('/content/daily_report.zip')